In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
!pip install -q openai #--progress-bar off

In [3]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [4]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [5]:
output_datapath = "data/output/2023.11.21-4_Llama2_13B_API_3-few-shot/"

In [6]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [7]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [8]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) #|
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000


In [9]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Output: "---SDG 12---"

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight again

In [10]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [11]:
template_part1 = """[INST]
<<SYS>>
In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 3 examples of expected classification given an input text:

"""+few_shots+"""

The text to be processed is:
<</SYS>>

'''
"""

template_part2 = """
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [18]:
data.iloc[181:183]

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
181,571,10.14217/9781848591448-10-en,Three states had child mortality rates of five...,3,0.0,3.0,1.000000
182,4948,10.1787/9789264283473-en,Patients need a referral to access specialist ...,3,1.0,8.0,0.777778


In [19]:
# data_sel = data #.sample(2)
data_sel = data.iloc[181:] #.sample(2)

In [13]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [20]:
import time

# cnt = 0
cnt = 181

results = []
for index, row in data_sel.iterrows():
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    query = template_part1 + text + template_part2
    
    body = {
      "model": "meta-llama/Llama-2-13b-chat-hf",
      "messages": [
          {"role": "system", "content": "You are a helpful assistant."}, 
          {"role": "user", "content": query}],
      "temperature": 0.0
    }
    
        
    try:
        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    except Exception as e:
        print("#############################################################################")
        print(e)
        print("#############################################################################")
        
    
    result_extended = (cnt, text_id, sdg_true, result)
    
    with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    time.sleep(0.5)

181 571 3   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 3---
182 4948 3   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 3---

The text mentions patients needing a referral to access specialist services for free, and the development of day care, day surgery, and outpatient rehabilitation services. These points relate to the goal of ensuring healthy lives and promoting well-being for all at all ages, which is SDG 3.
183 5101 3  Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 3---

The text you provided is related to the prevalence of mental disorders, which falls under the category of "Good Health and Well-being" (SDG 3).
184 2852 3   Sure, I'd be happy to help! Here's the classification of the given text according to the 

205 4023 3   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 3---
206 678 3   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 3---
207 4456 3   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 3---

The text you provided is related to the theme of "Good Health and Well-being" (SDG 3), specifically addressing the issue of underreporting of alcohol intake and its impact on population groups.
208 6027 3   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 11---

The text emphasizes the importance of improving the quality of life and health by ensuring a safe and sanitary living environment for all population groups, which is aligned with SDG 11: Susta

236 835 3   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 3---

The text you provided mentions "hospital care" and "primary care staff," which falls under the category of healthcare. SDG 3 focuses on good health and well-being, and the text you provided is directly related to this goal.
237 411 3   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 3---
238 4867 3   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 3---

The text refers to the redefinition of the scope of services provided by health care facilities and the need for adequate investment to cover the necessary physical and human resources. This is directly related to SDG 3, which focuses on ensuring healthy lives and promoting well-being for all at all ages.
239 69 3   Su

261 932 4   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 4---
262 6608 4   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 4---
263 1276 4  ---SDG 4---
264 2037 4  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 4---

The text discusses the assessment of teacher performance in decentralized countries like the United States, Germany, and Australia, and how it is defined by local authorities. This is related to SDG 4, which focuses on quality education.
265 5181 4  ---SDG 4---
266 2261 4   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 4---
267 6446 4   Sure, I'd be happy to help! Here's the classification of the given text according to the S

289 1960 4   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 4---

The text discusses the issue of quality in education, specifically the decrease in the percentage of qualified teachers, despite an increase in access to education. This is related to SDG 4, which aims to ensure inclusive and equitable quality education and promote lifelong learning for all.
290 4894 4   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 4---

The text focuses on improving the quality and relevance of education, which is a key aspect of SDG 4: Quality Education. The text also mentions the need to nurture talent at home and improve access to education for new immigrants, which aligns with the goal of ensuring equitable and inclusive education for all.
291 791 4   Sure, I'd be happy to help! Here's the classification of the text according to t

...

930 152 15   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 15---

The text mentions the overlapping of responsibilities and lack of coordination among different institutions, which affects the effective implementation of sectoral strategies and the protection of natural resources. This relates to SDG 15, which focuses on life on land.
931 658 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 15---
932 1672 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 15---
933 3170 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 15---
934 2084 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Deve

962 5813 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text focuses on the evolution of the narrative around guardianship as a crime prevention strategy, specifically in the context of residential places and property crime. The text also highlights the application of guardianship research beyond the residential context to interpersonal crimes and cybercrime. The discussion of these issues contributes to the goal of promoting peaceful and inclusive societies, access to justice for all, and effective, accountable, and transparent institutions (SDG 16).
963 5147 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses various aspects of politics and governance, including the role of elites, public opinion, and civil society, as well as the impact of globalization and transnationali

978 2543 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the interpretation and application of international humanitarian law, specifically in the context of the naval blockade of the Mavi Marmara. This is related to SDG 16, which focuses on promoting peaceful and inclusive societies, access to justice for all, and effective and accountable institutions.
979 64 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text focuses on the examination of the process unleashed by the adoption of the United Nations Security Council (UNSC) Resolution 1373 on the aftermath of the 9/11 attacks, and the factors that have constrained the "humanization" of the UNSC's counterterrorist discourse. This is related to SDG 16, which is focused on promoting peaceful and inclusive societies

997 322 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the topic of human rights and the civilizing process, with a focus on the impact of workplace bullying and morbid forms of solidarity on subjectivity. This topic is related to SDG 16, which is focused on promoting peaceful and inclusive societies, access to justice for all, and effective and accountable institutions.
998 6351 16  ---SDG 16---
999 4375 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 15---

The text discusses the impact of patents on access to science and its benefits, which is a fundamental human right enshrined in international law. This is related to SDG 15, which focuses on life on land.
1000 2522 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustain

1017 4563 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the accountability of government officials' actions and the potential for abuse of discretionary authority, which is related to the theme of peace, justice, and strong institutions (SDG 16).
1018 6710 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses legal issues related to class actions and due process concerns, specifically in the context of non-U.S. claimants participating in class litigation. This topic is relevant to SDG 16, which focuses on promoting peaceful and inclusive societies, access to justice for all, and effective and accountable institutions.
1019 1947 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals

In [21]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [22]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(865, 2639, 14, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):\n\n---SDG 14---\n\nThe text mentions the importance of considering social and economic sustainability in fisheries management practices, particularly in developing countries. This aligns with SDG 14, which aims to conserve and sustainably use the oceans, seas, and marine resources.")
(531, 6741, 8, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):\n\n---SDG 8---\n\nThe text discusses labor law enforcement and the dismissal of workers in India, which relates to SDG 8: Decent Work and Economic Growth.")
(256, 1972, 4, '  Sure, I\'d be happy to help! Here\'s the classification of the text you provided based on the Sustainable Development Goals (SDGs):\n\n---SDG 4---\n\nThe text mentions "school characteristics such as location and school size" and "socio-economically

In [23]:
len(tuple_list)

1020

In [24]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [25]:
results_processed

,cnt,id,true_label,predicted_label
0,865,2639,14,"Sure, I'd be happy to help! Here's the class..."
1,531,6741,8,"Sure, I'd be happy to help! Here's the class..."
2,256,1972,4,"Sure, I'd be happy to help! Here's the class..."
3,816,4186,13,"Sure, I'd be happy to help! Here's the class..."
4,444,2876,7,"Sure, I'd be happy to help! Here's the class..."
...,...,...,...,...
1015,487,433,8,"Sure, I'd be happy to help! Here's the class..."
1016,34,6228,0,"Sure, I'd be happy to help! Here's the label..."
1017,37,4226,0,Sure! Here's the label for the given text:\n...
1018,907,3261,15,"Sure, I'd be happy to help! Here's the class..."


In [26]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class..."
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the class..."
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class..."


In [27]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class...",11
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label...",4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---,16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class...",16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the class...",16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class...",16


In [28]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
1,66,66,66,66,66,66,66,66,66,66,66
10,31,30,31,31,30,30,30,31,31,31,31
11,73,65,73,73,65,65,65,73,73,73,73
12,68,57,68,68,57,57,57,68,68,68,68
13,58,57,58,58,57,57,57,58,58,58,58
14,42,42,42,42,42,42,42,42,42,42,42
15,79,76,79,79,76,76,76,79,79,79,79
16,69,60,69,69,60,60,60,69,69,69,69
17,24,23,24,24,23,23,23,24,24,24,24


In [29]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_13B.xlsx')

In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.70      0.77      0.73        60
           2       0.95      0.68      0.80        60
           3       0.70      0.92      0.79        60
           4       0.57      0.97      0.72        60
           5       0.85      0.85      0.85        60
           6       0.98      0.82      0.89        60
           7       0.91      0.68      0.78        60
           8       0.48      0.52      0.50        60
           9       0.59      0.65      0.62        60
          10       0.77      0.40      0.53        60
          11       0.60      0.73      0.66        60
          12       0.57      0.65      0.61        60
          13       0.76      0.73      0.75        60
          14       0.95      0.67      0.78        60
          15       0.71      0.93      0.81        60
          16       0.71      0.82      0.76        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.